In [1]:
import gspread
import numpy as np
import os
import pandas as pd
import pytz
import re
import requests
import unicodedata

from datetime import datetime
from dotenv import load_dotenv
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path

def carregar_excel(caminho_arquivo):
    caminho = Path(caminho_arquivo)
    if caminho.exists():
        try:
            xls = pd.ExcelFile(caminho, engine='openpyxl')
            print(f"[INFO] Abas encontradas em '{caminho.name}': {xls.sheet_names}")
            if not xls.sheet_names:
                print(f"[ERRO] Nenhuma aba encontrada em: {caminho.name}")
                return None
            df = xls.parse(xls.sheet_names[0])
            print(f"[OK] Carregado: {caminho.name}")
            return df
        except Exception as e:
            print(f"[ERRO] Falha ao carregar '{caminho.name}': {e}")
            return None
    else:
        print(f"[ERRO] Arquivo não encontrado: {caminho}")
        return None

def carregar_csv(caminho_arquivo):
    caminho = Path(caminho_arquivo)
    if not caminho.exists():
        print(f"[ERRO] Arquivo CSV não encontrado: {caminho}")
        return None

    for sep in [',', ';', '\t', '|']:
        try:
            df = pd.read_csv(caminho, sep=sep, engine='python')
            print(f"[OK] CSV carregado com separador '{sep}': {caminho.name}")
            return df
        except Exception as e:
            print(f"[INFO] Tentativa com separador '{sep}' falhou: {e}")

    print(f"[ERRO] Falha ao carregar CSV '{caminho.name}' com os separadores testados.")
    return None

# Caminho base seguro para script .py e notebooks
try:
    BASE_DIR = Path(__file__).resolve().parent
except NameError:
    BASE_DIR = Path().resolve().parent  # << sobe 1 nível

base_path = BASE_DIR / "dados"

# Carregar CSV de respostas da pesquisa
df_leads_antigos = carregar_csv(base_path / 'RESPOSTAS PESQUISA L28^0L29^0L30.csv')
df_compradores_antigos = carregar_csv(base_path / 'COMPRADORES L28,L29 E L30.csv')
df_utms_antigas = carregar_csv(base_path / 'LISTA LEADS UTM L28,L29 E L30.csv')

# Carregar lista de compradores
df_compradores = carregar_excel(base_path / 'lista alunos dez-24 a atual.xlsx')

# Carregar planilha de UTM
df_utms = carregar_excel(base_path / 'LISTA DE CADASTRO L31 - L32 - L33  COM UTM.xlsx')

# Carregar abas específicas do arquivo de leads
arquivo_leads = base_path / 'RESPOSTAS PESQUISAS ELEMENTOR E FORMS - COM DATA.xlsx'
if arquivo_leads.exists():
    try:
        xls_leads = pd.ExcelFile(arquivo_leads, engine='openpyxl')
        print(f"[INFO] Abas encontradas em '{arquivo_leads.name}': {xls_leads.sheet_names}")
        df_leads_google = xls_leads.parse('GoogleForms')
        df_leads_elementor = xls_leads.parse('ElementorForms')
        print(f"[OK] Abas 'GoogleForms' e 'ElementorForms' carregadas com sucesso.")
    except Exception as e:
        print(f"[ERRO] Falha ao carregar abas de leads: {e}")
        df_leads_google, df_leads_elementor = None, None
else:
    print(f"[ERRO] Arquivo de leads não encontrado: {arquivo_leads}")
    df_leads_google, df_leads_elementor = None, None

[INFO] Tentativa com separador ',' falhou: ',' expected after '"'
[OK] CSV carregado com separador ';': RESPOSTAS PESQUISA L28^0L29^0L30.csv
[OK] CSV carregado com separador ',': COMPRADORES L28,L29 E L30.csv
[INFO] Tentativa com separador ',' falhou: Expected 1 fields in line 14, saw 3
[OK] CSV carregado com separador ';': LISTA LEADS UTM L28,L29 E L30.csv
[INFO] Abas encontradas em 'lista alunos dez-24 a atual.xlsx': ['users_csv (6)']
[OK] Carregado: lista alunos dez-24 a atual.xlsx
[INFO] Abas encontradas em 'LISTA DE CADASTRO L31 - L32 - L33  COM UTM.xlsx': ['LISTA LEADTRACKER COM UTM']
[OK] Carregado: LISTA DE CADASTRO L31 - L32 - L33  COM UTM.xlsx
[INFO] Abas encontradas em 'RESPOSTAS PESQUISAS ELEMENTOR E FORMS - COM DATA.xlsx': ['GoogleForms', 'ElementorForms']
[OK] Abas 'GoogleForms' e 'ElementorForms' carregadas com sucesso.


# Tratamento Compradores

In [2]:
df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Nome             849 non-null    object        
 1   E-mail           849 non-null    object        
 2   Papel            849 non-null    object        
 3   Acesso           849 non-null    object        
 4   Turma            849 non-null    object        
 5   Categoria        849 non-null    object        
 6   Primeiro acesso  806 non-null    datetime64[ns]
 7   Ãšltimo acesso   806 non-null    datetime64[ns]
 8   Progresso        849 non-null    float64       
 9   Engajamento      849 non-null    object        
 10  Data da compra   849 non-null    datetime64[ns]
 11  NÂº de acessos   849 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(1), object(7)
memory usage: 79.7+ KB


In [3]:
df_compradores.columns = df_compradores.columns.str.lower()

df_compradores.rename(columns={'e-mail': 'email'}, inplace=True)
df_compradores.rename(columns={'primeiro acesso': 'primeiro_acesso'}, inplace=True)
df_compradores.rename(columns={'ãšltimo acesso': 'ultimo_acesso'}, inplace=True)
df_compradores.rename(columns={'nâº de acessos': 'numero_acesso'}, inplace=True)
df_compradores.rename(columns={'data da compra': 'data'}, inplace=True)


df_compradores["email"] = df_compradores["email"].str.lower().str.strip()
df_compradores = df_compradores.drop_duplicates(subset='email', keep='first')

df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 849 entries, 0 to 848
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   nome             849 non-null    object        
 1   email            849 non-null    object        
 2   papel            849 non-null    object        
 3   acesso           849 non-null    object        
 4   turma            849 non-null    object        
 5   categoria        849 non-null    object        
 6   primeiro_acesso  806 non-null    datetime64[ns]
 7   ultimo_acesso    806 non-null    datetime64[ns]
 8   progresso        849 non-null    float64       
 9   engajamento      849 non-null    object        
 10  data             849 non-null    datetime64[ns]
 11  numero_acesso    849 non-null    int64         
dtypes: datetime64[ns](3), float64(1), int64(1), object(7)
memory usage: 79.7+ KB


In [4]:
df_compradores.drop(columns=['email', 'nome']).head()

,papel,acesso,turma,categoria,primeiro_acesso,ultimo_acesso,progresso,engajamento,data,numero_acesso
0,Estudante,Ativo,Turma A,Comprador,2024-12-11,2025-05-06,20.0,Alto,2024-12-11,233
1,Estudante,Ativo,Turma A,Importado,2025-02-05,2025-03-22,0.0,Nenhum,2025-03-05,5
2,Estudante,Ativo,Turma A,Comprador,2025-02-10,2025-03-25,0.0,Baixo,2025-02-10,5
3,Estudante,Ativo,Turma A,Comprador,2024-12-10,2025-02-16,3.0,Baixo,2024-12-09,30
4,Estudante,Ativo,Turma A,Comprador,2025-03-31,2025-03-31,0.0,Nenhum,2025-02-10,1


In [5]:
# Separa a coluna 'LANÇAMENTO;EMAIL' em duas novas colunas
df_compradores_antigos[['LANÇAMENTO', 'EMAIL']] = df_compradores_antigos['LANÇAMENTO;EMAIL'].str.split(';', expand=True)

# (Opcional) Remove a coluna original
df_compradores_antigos.drop(columns=['LANÇAMENTO;EMAIL'], inplace=True)

In [6]:
df_compradores_antigos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 647 entries, 0 to 646
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   LANÇAMENTO  647 non-null    object
 1   EMAIL       647 non-null    object
dtypes: object(2)
memory usage: 10.2+ KB


In [7]:
df_compradores_antigos.columns = df_compradores_antigos.columns.str.lower()

df_compradores_antigos.rename(columns={'EMAIL': 'email'}, inplace=True)
df_compradores_antigos.rename(columns={'LANÇAMENTO': 'lancamentos'}, inplace=True)

df_compradores_antigos["email"] = df_compradores_antigos["email"].str.lower().str.strip()
df_compradores_antigos = df_compradores_antigos.drop_duplicates(subset='email', keep='first')

df_compradores_antigos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 636 entries, 0 to 645
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lançamento  636 non-null    object
 1   email       636 non-null    object
dtypes: object(2)
memory usage: 14.9+ KB


In [8]:
df_compradores = pd.concat([df_compradores, df_compradores_antigos], ignore_index=True)

In [9]:
df_compradores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1485 entries, 0 to 1484
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   nome             849 non-null    object        
 1   email            1485 non-null   object        
 2   papel            849 non-null    object        
 3   acesso           849 non-null    object        
 4   turma            849 non-null    object        
 5   categoria        849 non-null    object        
 6   primeiro_acesso  806 non-null    datetime64[ns]
 7   ultimo_acesso    806 non-null    datetime64[ns]
 8   progresso        849 non-null    float64       
 9   engajamento      849 non-null    object        
 10  data             849 non-null    datetime64[ns]
 11  numero_acesso    849 non-null    float64       
 12  lançamento       636 non-null    object        
dtypes: datetime64[ns](3), float64(2), object(8)
memory usage: 150.9+ KB


# Tratamento Leads

In [10]:
df_leads_antigos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4046 entries, 0 to 4045
Data columns (total 13 columns):
 #   Column                                                                                                                Non-Null Count  Dtype 
---  ------                                                                                                                --------------  ----- 
 0   Carimbo de data/hora                                                                                                  4046 non-null   object
 1   LANÇAMENTO                                                                                                            4046 non-null   object
 2   Nome completo                                                                                                         3684 non-null   object
 3   Seu melhor e-mail                                                                                                     3684 non-null   object
 4   Número de Wh

In [11]:
colunas_renomeadas = {
    'Carimbo de data/hora': 'data',
    'LANÇAMENTO': 'lancamentos',
    'Seu melhor e-mail': 'email',
    'Nome completo': 'nome',
    'Número de WhatsApp': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'Qual seu estado civil?': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'Por que você escolheu/escolheria a profissão de Policial Civil?': 'escolheu_profissao',
    'O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?': 'dificuldade'
}

df_leads_antigos.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_antigos.columns:
    df_leads_antigos['email'] = df_leads_antigos['email'].astype(str).str.lower().str.strip()
    df_leads_antigos = df_leads_antigos.drop_duplicates(subset=['email', 'lancamentos'])

# Exibir resumo
df_leads_antigos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3501 entries, 0 to 4045
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                3501 non-null   object
 1   lancamentos         3501 non-null   object
 2   nome                3500 non-null   object
 3   email               3501 non-null   object
 4   whatsapp            3500 non-null   object
 5   estado              3500 non-null   object
 6   idade               3500 non-null   object
 7   escolaridade        3500 non-null   object
 8   renda               3500 non-null   object
 9   estado_civil        3500 non-null   object
 10  filhos              3500 non-null   object
 11  escolheu_profissao  3501 non-null   object
 12  dificuldade         3501 non-null   object
dtypes: object(13)
memory usage: 382.9+ KB


In [12]:
df_leads_antigos["renda"] = (
    df_leads_antigos["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [13]:
# Normalize o dicionário para garantir que as chaves estejam em minúsculo
estado_siglas = {
    k.lower(): v for k, v in {
        'são paulo': 'SP',
        'sp': 'SP',
        'rio de janeiro': 'RJ',
        'rj': 'RJ',
        'minas gerais': 'MG',
        'mg': 'MG',
        'bahia': 'BA',
        'ba': 'BA',
        'goiás': 'GO',
        'goias': 'GO',
        'go': 'GO',
        'paraná': 'PR',
        'pr': 'PR',
        'espírito santo': 'ES',
        'es': 'ES',
        'rio grande do sul': 'RS',
        'rs': 'RS',
        'santa catarina': 'SC',
        'sc': 'SC',
        'alagoas': 'AL',
        'al': 'AL',
        'paraíba': 'PB',
        'paraiba': 'PB',
        'pb': 'PB',
        'pará': 'PA',
        'pa': 'PA',
        'maranhão': 'MA',
        'ma': 'MA',
        'pernambuco': 'PE',
        'pe': 'PE',
        'amazonas': 'AM',
        'am': 'AM',
        'rio grande do norte': 'RN',
        'rn': 'RN',
        'distrito federal': 'DF',
        'df': 'DF',
        'boa vista': 'RR',
        'rr': 'RR',
        'ceará': 'CE',
        'brasília': 'DF',
        'brasilia': 'DF',
        'florianopolis': 'SC',
        'guarulhos': 'SP',
        'guarujá': 'SP',
        'Rio das ostras': 'RJ',
        'Granja': 'RJ',
        'Ceara': 'CE',
        'Tocantins': 'TO',
        'Belém': 'PA',
        'Acre': 'AC',
        'Rondônia': 'RO',
        'Itanhaém': 'SP',
        'Rio de janriro': 'RJ',
        'Volta redonda': 'RJ',
        'Botucatu': 'SP',
        'Mogi mirim': 'SP',
        'Feira de Santana': 'BA',
        'Recife': 'PE'
    }.items()
}

# Função de normalização
def normalizar_estado(valor):
    if pd.isna(valor):
        return None
    texto = str(valor).lower().strip()
    for nome, sigla in estado_siglas.items():
        if nome in texto:
            return sigla
    return valor

# Aplicar a função
df_leads_antigos['estado'] = df_leads_antigos['estado'].apply(normalizar_estado)

print(df_leads_antigos['estado'].value_counts(dropna=False))

estado
SP      2882
RJ       319
MG       229
BA        15
MA        15
PR         7
GO         6
SC         5
DF         4
RS         4
CE         4
RN         2
AL         2
None       1
AC         1
AM         1
PA         1
RO         1
RR         1
ES         1
Name: count, dtype: int64


In [14]:
# Categorias permitidas
CATEGORIAS_PERMITIDAS = {
    'médio completo',
    'superior completo',
    'fundamental completo',
    'superior incompleto',
    'técnico',
    'médio incompleto',
    'fundamental incompleto'
}

# 1. Pré-processamento
def limpar_texto(texto):
    texto = str(texto).lower().strip()
    return unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')

# 2. Substituições comuns
SUBSTITUICOES = {
    'imcompleto': 'incompleto',
    'inclopeto': 'incompleto',
    'incompleta': 'incompleto',
    'enfetmagem': 'enfermagem',
    'encino': 'ensino',
    'esino': 'ensino',
    'ensaio': 'ensino',
    'ensino superior incomoda': 'superior incompleto',
    'sup incompleto mais cursando': 'superior incompleto',
    'superior curso': 'superior incompleto',
    'superior engenharia mecanica incompleto': 'superior incompleto',
    'comecei direito, mas nao finalizei': 'superior incompleto',
    'faco radiologia': 'superior incompleto',
}

def aplicar_substituicoes(val):
    for k, v in SUBSTITUICOES.items():
        val = val.replace(k, v)
    return val

# 3. Mapeamento explícito
MAPEAMENTO_EXPLICITO = {
    'incompleto': 'superior incompleto',
    'none': None,
    'cursando': 'superior incompleto',
    'setimo ano': 'fundamental incompleto',
    'comecando a faculdade': 'superior incompleto',
    'cursando terceiro ano medio': 'médio incompleto',
    'bombeira civil': 'técnico',
    'sim': None,
    'auxiliar de enfermagem': 'técnico',
    'enfermagem': 'superior incompleto',
    '5 seria': 'fundamental incompleto',
    '8 seria': 'fundamental incompleto',
    'estudante': 'médio incompleto',
    'pos graduado': 'superior completo',
    'estou no 2°ano ensino medio': 'médio incompleto',
    'e completo': 'médio completo',
    'cursando pedagogia': 'superior incompleto',
    'cursando gestao em rh': 'superior incompleto',
    "2'ano": 'fundamental incompleto',
    'tec enfermagem': 'técnico',
    'ciencias contabeis': 'superior incompleto',
    'ensino medio completando': 'médio incompleto',
    '2': 'fundamental incompleto',
    'cursando o supervisor': 'superior incompleto',
    'estudar ainda': 'médio incompleto',
    'cursando gestao agronegocio': 'superior incompleto',
    'terapeuta ocupacional ( cursando)': 'superior incompleto',
    'engenharia nao concluida': 'superior incompleto',
    'preciso terminar o ensino fundamental': 'fundamental incompleto',
    'alguns certificados na area da seguranca': 'técnico',
}

def aplicar_mapeamento_explicito(val):
    return MAPEAMENTO_EXPLICITO.get(val)

# 4. Categorização padrão
def categorizar_padrao(val):
    if any(p in val for p in ['fundamental incompleto', 'parei no', 'nao terminei fundamental', 'nono ano', 'quint', 'sere', 'serie']) \
       or re.search(r'\b[1-9]{1,2}[ºo]?\s*(ano|serie)', val) and 'medio' not in val:
        return 'fundamental incompleto'
    
    if 'fundamental completo' in val or 'primeiro grau completo' in val:
        return 'fundamental completo'

    if any(p in val for p in ['ensino medio incompleto', 'medio incompleto', '2º colegial incompleto', 'eja', 'ensino medio nao']):
        return 'médio incompleto'

    if any(p in val for p in ['ensino medio completo', 'medio completo', 'magisterio']):
        return 'médio completo'

    if any(p in val for p in ['superior completo', 'graduacao completa', 'nivel superior completo', 'pos-graduacao', 'pos graduacao']) \
       or ('graduacao' in val and 'cursando' not in val and 'incompleto' not in val):
        return 'superior completo'

    if any(p in val for p in ['tecnico', 'tecnologo', 'tecnologia', 'curso tec']):
        return 'técnico'

    if any(v in val for v in ['cursando', 'concluindo', 'fazendo', 'em andamento', 'graduando', 'estudando', 'terminando']):
        if any(n in val for n in ['superior', 'faculdade', 'universidade', 'direito', 'biomedicina', 'administracao']):
            return 'superior incompleto'
        elif 'ensino medio' in val or 'medio' in val or 'colegial' in val:
            return 'médio incompleto'
        elif 'fundamental' in val or 'serie' in val:
            return 'fundamental incompleto'

    if re.search(r'[0-9]{1,2}.*semestre', val) or 'periodo' in val:
        return 'superior incompleto'

    return val

# 5. Ajustes finais
AJUSTES_FINAIS = {
    'pos graduado': 'superior completo',
    'pos graduada': 'superior completo',
    'pos-graduacao': 'superior completo',
    'pos-graduada': 'superior completo',
    'pos graduado em gestao de projetos': 'superior completo',
    'mestrado': 'superior completo',
    'mba': 'superior completo',
    'cursando graduacao': 'superior incompleto',
    'cursando graduacao em radiologia': 'superior incompleto',
    'cursando graduacao em enfermagem': 'superior incompleto',
    'cursanso ensino superior': 'superior incompleto',
    'cursanso faculdade': 'superior incompleto',
    'superior incompleto ☹️': 'superior incompleto',
    'estou, termiando o ensino medio!': 'médio incompleto',
    '8 incompleto': 'fundamental incompleto',
}

def aplicar_ajustes_finais(val):
    return AJUSTES_FINAIS.get(val, val)

# --------------------------
# Função principal
# --------------------------
def normalizar_escolaridade(valor):
    if pd.isna(valor):
        return None

    val = limpar_texto(valor)
    val = aplicar_substituicoes(val)

    mapeado = aplicar_mapeamento_explicito(val)
    if mapeado:
        val = mapeado
    else:
        val = categorizar_padrao(val)

    val = aplicar_ajustes_finais(val)

    if val not in CATEGORIAS_PERMITIDAS:
        return None

    return val

In [15]:
df_leads_antigos['escolaridade'] = df_leads_antigos['escolaridade'].apply(normalizar_escolaridade)

df_leads_antigos['escolaridade'].value_counts()

escolaridade
médio completo            2310
superior completo          584
fundamental completo       262
superior incompleto        154
médio incompleto            41
técnico                     38
fundamental incompleto      19
Name: count, dtype: int64

In [16]:
# Função de limpeza de texto
def limpar_texto(texto):
    texto = str(texto).strip().lower()
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = re.sub(r'[^\w\s]', '', texto)  # remove pontuação
    return texto

# Mapeamento por categoria
def normalizar_estado_civil(valor):
    if pd.isna(valor):
        return None

    val = limpar_texto(valor)

    if val in {'solteiroa', 'solteiro', 'viuvo solteiro'}:
        return 'solteiro(a)'

    if val in {'casadoa', 'tenho esposa e filhos mais nao casado formalmente'}:
        return 'casado(a)'

    if val in {
        'divorciadoa ou separadora', 'divorciadoa', 'separado', 'separada', 'em processo de separacao',
        'divorciando', 'divorciado uniao estavel'
    }:
        return 'divorciado(a)'

    if val in {
        'uniao estavel', 'moro junto', 'mora junto', 'moramos juntos', 'morando junto',
        'morando com alguem', 'convivente', 'amigado', 'amigada', 'amaziado',
        'moro com um pessoal', 'tenho companheira e filhos', 'moro a 20 anos',
        'moro a mais de 5 anos com minha conjuge', 'ajuntado sem uniao estavel', 'namorando'
    } or 'moro junto' in val or 'com meu parceiro' in val:
        return 'união estável'

    if val in {'viuva', 'viuvo', 'viuva de companheiro'}:
        return 'viúvo(a)'

    return valor  # mantém valor original se não reconhecido

# Ajustes finais de exceções residuais
AJUSTES_FINAIS_ESTADO_CIVIL = {
    'vendendor autonomo e sou vigilante': None,
    'estou terminado o ensino medio': None,
    '5serie': None,
    'enrolada': 'união estável',
    'leandro alves': None
}

def aplicar_ajustes_finais_estado_civil(valor):
    if valor in CATEGORIAS_VALIDAS_ESTADO_CIVIL:
        return valor  # já está padronizado

    val_limpo = limpar_texto(valor)
    return AJUSTES_FINAIS_ESTADO_CIVIL.get(val_limpo, None)  # se não reconhecido, descarta

# Categorias válidas
CATEGORIAS_VALIDAS_ESTADO_CIVIL = {
    'solteiro(a)',
    'casado(a)',
    'divorciado(a)',
    'união estável',
    'viúvo(a)'
}

# Aplicação no DataFrame
df_leads_antigos['estado_civil'] = df_leads_antigos['estado_civil'].apply(normalizar_estado_civil)
df_leads_antigos['estado_civil'] = df_leads_antigos['estado_civil'].apply(aplicar_ajustes_finais_estado_civil)

# Filtrar apenas os contatos com estado civil válido
df_leads_antigos = df_leads_antigos[df_leads_antigos['estado_civil'].isin(CATEGORIAS_VALIDAS_ESTADO_CIVIL)]

df_leads_antigos['estado_civil'].value_counts()

estado_civil
solteiro(a)      1350
casado(a)        1278
união estável     441
viúvo(a)           19
divorciado(a)       1
Name: count, dtype: int64

In [17]:
# Categorias válidas (respostas simples)
CATEGORIAS_VALIDAS_PROFISSAO = {
    'sonho de criança',
    'gosta da profissão',
    'estabilidade de emprego',
    'prestígio da carreira',
    'segurança'
}

# Função de normalização
def limpar_profissao_simples(valor):
    if pd.isna(valor):
        return None

    val = str(valor).strip().lower().strip("., ")

    # Se for exatamente uma das categorias válidas, mantém
    if val in CATEGORIAS_VALIDAS_PROFISSAO:
        return val

    # Caso contrário, descarta
    return None

# Aplicação no DataFrame
df_leads_antigos['escolheu_profissao'] = df_leads_antigos['escolheu_profissao'].apply(limpar_profissao_simples)
df_leads_antigos = df_leads_antigos[df_leads_antigos['escolheu_profissao'].isin(CATEGORIAS_VALIDAS_PROFISSAO)]

df_leads_antigos['escolheu_profissao'].value_counts()

escolheu_profissao
sonho de criança           744
gosta da profissão         650
prestígio da carreira      241
estabilidade de emprego    220
segurança                   64
Name: count, dtype: int64

In [18]:
# Categorias válidas para dificuldade
DIFICULDADES_VALIDAS = {
    'financeiro / dinheiro',
    'falta de tempo',
    'falta de base escolar',
    'falta de oportunidade',
    'idade'
}

# Função de limpeza e normalização
def limpar_dificuldade_simples(valor):
    if pd.isna(valor):
        return None

    val = str(valor).strip().lower().strip("., ")

    if val in DIFICULDADES_VALIDAS:
        return val
    return None

df_leads_antigos['dificuldade'] = df_leads_antigos['dificuldade'].apply(limpar_dificuldade_simples)
df_leads_antigos = df_leads_antigos[df_leads_antigos['dificuldade'].isin(DIFICULDADES_VALIDAS)]

df_leads_antigos['dificuldade'].value_counts()

dificuldade
financeiro / dinheiro    608
falta de oportunidade    569
falta de tempo           306
idade                     70
Name: count, dtype: int64

In [19]:
df_leads_antigos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1553 entries, 1 to 4042
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                1553 non-null   object
 1   lancamentos         1553 non-null   object
 2   nome                1553 non-null   object
 3   email               1553 non-null   object
 4   whatsapp            1553 non-null   object
 5   estado              1553 non-null   object
 6   idade               1553 non-null   object
 7   escolaridade        1517 non-null   object
 8   renda               1553 non-null   object
 9   estado_civil        1553 non-null   object
 10  filhos              1553 non-null   object
 11  escolheu_profissao  1553 non-null   object
 12  dificuldade         1553 non-null   object
dtypes: object(13)
memory usage: 169.9+ KB


In [20]:
colunas_excluir = ["nome", "data", "email", "whatsapp"]

for coluna in df_leads_antigos.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_antigos[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L30    605
L29    495
L28    453
Name: count, dtype: int64

Coluna: estado
estado
SP    1255
RJ     138
MG     129
BA       8
MA       7
PR       3
GO       2
RS       2
SC       2
DF       2
AC       1
RO       1
AM       1
AL       1
RN       1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        615
26 - 35 anos        496
46 - 55 anos        319
Até 25 anos         103
Acima de 56 anos     20
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            1061
superior completo          240
fundamental completo       121
superior incompleto         57
None                        36
técnico                     16
médio incompleto            12
fundamental incompleto      10
Name: count, dtype: int64

Coluna: renda
renda
De 1.000 a 3.000    840
De 3.000 a 5.000    308
Até 1.000           164
Desempregado        152
Acima de 5.000       89
Name: count, dtype: int64

Coluna: estado_civil
estado_civil
solteiro(a)     

In [21]:
df_leads_google.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4800 entries, 0 to 4799
Data columns (total 14 columns):
 #   Column                                                                                                                Non-Null Count  Dtype         
---  ------                                                                                                                --------------  -----         
 0   Data                                                                                                                  4800 non-null   datetime64[ns]
 1   LANÇAMENTO                                                                                                            4800 non-null   object        
 2   Endereço de e-mail                                                                                                    4800 non-null   object        
 3   Nome completo                                                                                                         

In [22]:
# Renomear colunas do df_leads_google
colunas_renomeadas = {
    'Data': 'data',
    'LANÇAMENTO': 'lancamentos',
    'Endereço de e-mail': 'email',
    'Nome completo': 'nome',
    'Número de WhatsApp': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'Qual seu estado civil?': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'Por que você escolheu/escolheria a profissão de Policial Civil?': 'escolheu_profissao',
    'O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?': 'dificuldade'
}

df_leads_google.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_google.columns:
    df_leads_google['email'] = df_leads_google['email'].astype(str).str.lower().str.strip()
    df_leads_google = df_leads_google.drop_duplicates(subset=['email', 'lancamentos'])

# Exibir resumo
df_leads_google.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4450 entries, 0 to 4799
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                4450 non-null   datetime64[ns]
 1   lancamentos         4450 non-null   object        
 2   email               4450 non-null   object        
 3   nome                4450 non-null   object        
 4   Seu melhor e-mail   4450 non-null   object        
 5   whatsapp            4450 non-null   object        
 6   estado              4450 non-null   object        
 7   idade               4450 non-null   object        
 8   escolaridade        4450 non-null   object        
 9   renda               4450 non-null   object        
 10  estado_civil        4450 non-null   object        
 11  filhos              4450 non-null   object        
 12  escolheu_profissao  4450 non-null   object        
 13  dificuldade         4449 non-null   object        
dt

In [23]:
df_leads_google.drop(columns=['Seu melhor e-mail', 'nome'], inplace=True, errors='ignore')

In [24]:
df_leads_google["renda"] = (
    df_leads_google["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [25]:
df_leads_google['estado'] = df_leads_google['estado'].apply(normalizar_estado)

df_leads_google['estado'].value_counts()

estado
SP    3423
RJ     541
MG     377
BA      20
GO      20
MA      18
PR      14
SC       7
RS       6
DF       4
PA       4
CE       4
ES       3
PE       3
AL       3
RR       2
RN       1
Name: count, dtype: int64

In [26]:
df_leads_google['escolaridade'] = df_leads_google['escolaridade'].apply(normalizar_escolaridade)

df_leads_google['escolaridade'].value_counts()

escolaridade
médio completo            3059
superior completo          731
fundamental completo       320
superior incompleto        171
técnico                     61
fundamental incompleto      29
médio incompleto            28
Name: count, dtype: int64

In [27]:
df_leads_google['estado_civil'] = df_leads_google['estado_civil'].apply(normalizar_estado_civil)
df_leads_google['estado_civil'] = df_leads_google['estado_civil'].apply(aplicar_ajustes_finais_estado_civil)
df_leads_google = df_leads_google[df_leads_google['estado_civil'].isin(CATEGORIAS_VALIDAS_ESTADO_CIVIL)]

df_leads_google['estado_civil'].value_counts()

estado_civil
casado(a)        1679
solteiro(a)      1660
união estável     509
viúvo(a)           20
divorciado(a)       5
Name: count, dtype: int64

In [28]:
df_leads_google['escolheu_profissao'] = df_leads_google['escolheu_profissao'].apply(limpar_profissao_simples)
df_leads_google = df_leads_google[df_leads_google['escolheu_profissao'].isin(CATEGORIAS_VALIDAS_PROFISSAO)]

df_leads_google['escolheu_profissao'].value_counts()

escolheu_profissao
sonho de criança           873
gosta da profissão         826
estabilidade de emprego    378
prestígio da carreira      235
segurança                   83
Name: count, dtype: int64

In [29]:
df_leads_google['dificuldade'] = df_leads_google['dificuldade'].apply(limpar_dificuldade_simples)
df_leads_google = df_leads_google[df_leads_google['dificuldade'].isin(DIFICULDADES_VALIDAS)]

df_leads_google['dificuldade'].value_counts()

dificuldade
financeiro / dinheiro    769
falta de tempo           502
falta de base escolar    319
falta de oportunidade    203
idade                    162
Name: count, dtype: int64

In [30]:
df_leads_google.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade
11,2025-03-17 10:41:52,L33,RJ,Acima de 56 anos,fundamental completo,Até 1.000,casado(a),Sim,gosta da profissão,idade
12,2025-03-17 10:42:16,L33,SP,Acima de 56 anos,médio completo,De 3.000 a 5.000,casado(a),Sim,gosta da profissão,falta de base escolar
15,2025-03-17 10:44:49,L33,MG,36 - 45 anos,médio completo,Até 1.000,solteiro(a),Sim,estabilidade de emprego,financeiro / dinheiro
16,2025-03-17 10:50:45,L33,SP,36 - 45 anos,médio completo,De 1.000 a 3.000,casado(a),Sim,gosta da profissão,financeiro / dinheiro
18,2025-03-17 11:07:02,L33,SP,26 - 35 anos,fundamental completo,De 1.000 a 3.000,solteiro(a),Sim,gosta da profissão,falta de base escolar


In [31]:
df_leads_google.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1955 entries, 11 to 4797
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                1955 non-null   datetime64[ns]
 1   lancamentos         1955 non-null   object        
 2   email               1955 non-null   object        
 3   whatsapp            1955 non-null   object        
 4   estado              1955 non-null   object        
 5   idade               1955 non-null   object        
 6   escolaridade        1935 non-null   object        
 7   renda               1955 non-null   object        
 8   estado_civil        1955 non-null   object        
 9   filhos              1955 non-null   object        
 10  escolheu_profissao  1955 non-null   object        
 11  dificuldade         1955 non-null   object        
dtypes: datetime64[ns](1), object(11)
memory usage: 198.6+ KB


In [32]:
df_leads_elementor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24349 entries, 0 to 24348
Data columns (total 12 columns):
 #   Column                                                                                                                Non-Null Count  Dtype         
---  ------                                                                                                                --------------  -----         
 0   Unnamed: 0                                                                                                            24349 non-null  datetime64[ns]
 1   LANÇAMENTO                                                                                                            24349 non-null  object        
 2   Endereço de e-mail                                                                                                    24349 non-null  object        
 3   Número de WhatsApp                                                                                                  

In [33]:
colunas_renomeadas = {
    'Unnamed: 0': 'data',
    'LANÇAMENTO': 'lancamentos',
    'Endereço de e-mail': 'email',
    'Nome completo': 'nome',
    'Número de WhatsApp': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'Qual seu estado civil?': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'Por que você escolheu/escolheria a profissão de Policial Civil?': 'escolheu_profissao',
    'O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?': 'dificuldade'
}

df_leads_elementor.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_elementor.columns:
    df_leads_elementor['email'] = df_leads_elementor['email'].astype(str).str.lower().str.strip()
    df_leads_elementor = df_leads_elementor.drop_duplicates(subset=['email', 'lancamentos'])

# Exibir resumo
df_leads_elementor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22783 entries, 0 to 24348
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                22783 non-null  datetime64[ns]
 1   lancamentos         22783 non-null  object        
 2   email               22783 non-null  object        
 3   whatsapp            20902 non-null  object        
 4   estado              22407 non-null  object        
 5   idade               21872 non-null  object        
 6   escolaridade        22367 non-null  object        
 7   renda               22472 non-null  object        
 8   estado_civil        22563 non-null  object        
 9   filhos              22658 non-null  object        
 10  escolheu_profissao  22683 non-null  object        
 11  dificuldade         21139 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 2.3+ MB


In [34]:
df_leads_elementor["renda"] = (
    df_leads_elementor["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [35]:
colunas_excluir = ["data", "email", "whatsapp", "nome"]

for coluna in df_leads_elementor.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_elementor[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L32    10561
L33     7126
L31     5096
Name: count, dtype: int64

Coluna: estado
estado
São Paulo             16403
Rio de Janeiro         3038
Minas Gerais           2225
Outro                   575
NaN                     376
Paraná                   92
Mato Grosso do Sul       74
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        8051
26 - 35 anos        7278
46 - 55 anos        4245
Até 25 anos         1952
NaN                  911
Acima de 56 anos     346
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino Médio Completo                                                                  15419
Ensino Superior Completo                                                                3592
Ensino Fundamental Completo                                                             1540
Outro                                                                                   1070
NaN                                                  

In [36]:
df_leads_elementor['estado'] = df_leads_elementor['estado'].apply(normalizar_estado)

df_leads_elementor['estado'].value_counts()

estado
SP       16403
RJ        3038
MG        2225
Outro      575
PR          92
MA          74
Name: count, dtype: int64

In [37]:
df_leads_elementor['escolaridade'] = df_leads_elementor['escolaridade'].apply(normalizar_escolaridade)

df_leads_elementor['escolaridade'].value_counts()

escolaridade
médio completo          15735
superior completo        3649
fundamental completo     1913
Name: count, dtype: int64

In [38]:
# -------------------------------------
# Lista de rendas válidas (minúsculas)
# -------------------------------------
RENDAS_VALIDAS = {
    'até 1.000',
    'de 1.000 a 3.000',
    'de 3.000 a 5.000',
    'acima de 5.000',
    'desempregado'
}

# -------------------------------------
# Função de normalização
# -------------------------------------
def normalizar_renda(valor):
    if pd.isna(valor):
        return None
    val = str(valor).strip().lower()
    return val if val in RENDAS_VALIDAS else None

# -------------------------------------
# Aplicação no DataFrame
# -------------------------------------
df_leads_elementor['renda_original'] = df_leads_elementor['renda']
df_leads_elementor['renda'] = df_leads_elementor['renda'].apply(normalizar_renda)

# -------------------------------------
# Filtrar apenas rendas válidas
# -------------------------------------
df_leads_elementor = df_leads_elementor[df_leads_elementor['renda'].isin(RENDAS_VALIDAS)]

# -------------------------------------
# Resultado final
# -------------------------------------
with pd.option_context('display.max_rows', None):
    print(df_leads_elementor['renda'].value_counts(dropna=False))

renda
de 1.000 a 3.000    11664
de 3.000 a 5.000     5168
até 1.000            2117
desempregado         1875
acima de 5.000       1607
Name: count, dtype: int64


In [39]:
# -------------------------------------
# Lista de valores válidos (minúsculos)
# -------------------------------------
ESTADO_CIVIL_VALIDOS = {
    'solteiro(a)',
    'casado(a)',
    'divorciado(a) ou separado(a)',
    'união estável'
}

# -------------------------------------
# Função de normalização
# -------------------------------------
def normalizar_estado_civil(valor):
    if pd.isna(valor):
        return None
    val = str(valor).strip().lower()
    return val if val in ESTADO_CIVIL_VALIDOS else None

# -------------------------------------
# Aplicar no DataFrame
# -------------------------------------
df_leads_elementor['estado_civil_original'] = df_leads_elementor['estado_civil'].apply(normalizar_estado_civil)
df_leads_elementor['estado_civil'] = df_leads_elementor['estado_civil'].apply(normalizar_estado_civil)

# -------------------------------------
# Filtrar apenas valores válidos
# -------------------------------------
df_leads_elementor = df_leads_elementor[df_leads_elementor['estado_civil'].isin(ESTADO_CIVIL_VALIDOS)]


with pd.option_context('display.max_rows', None):
    print(df_leads_elementor['estado_civil'].value_counts(dropna=False))

estado_civil
solteiro(a)                     9100
casado(a)                       7787
divorciado(a) ou separado(a)    2767
união estável                   2152
Name: count, dtype: int64


In [40]:
# -------------------------------------
# Valores válidos permitidos
# -------------------------------------
VALORES_FILHOS_VALIDOS = {'sim', 'não'}

# -------------------------------------
# Função de limpeza
# -------------------------------------
def normalizar_filhos(valor):
    if pd.isna(valor):
        return None
    val = str(valor).strip().lower()
    return val if val in VALORES_FILHOS_VALIDOS else None

# -------------------------------------
# Aplicar transformação
# -------------------------------------
df_leads_elementor['filhos_original'] = df_leads_elementor['filhos']
df_leads_elementor['filhos'] = df_leads_elementor['filhos'].apply(normalizar_filhos)

# -------------------------------------
# Filtrar apenas valores válidos
# -------------------------------------
df_leads_elementor = df_leads_elementor[df_leads_elementor['filhos'].isin(VALORES_FILHOS_VALIDOS)]

# -------------------------------------
# Verificar resultado final
# -------------------------------------
with pd.option_context('display.max_rows', None):
    print(df_leads_elementor['filhos'].value_counts(dropna=False))

filhos
sim    15272
não     6473
Name: count, dtype: int64


In [41]:
# -------------------------------------
# Categorias simples válidas
# -------------------------------------
MOTIVOS_VALIDOS = {
    'sonho de criança',
    'gosta da profissão',
    'estabilidade de emprego',
    'prestígio da carreira'
}

# -------------------------------------
# Função para normalizar
# -------------------------------------
def normalizar_escolheu_profissao(valor):
    if pd.isna(valor):
        return None
    val = str(valor).strip().lower()
    return val if val in MOTIVOS_VALIDOS else None

# -------------------------------------
# Aplicar ao DataFrame
# -------------------------------------
df_leads_elementor['escolheu_profissao_original'] = df_leads_elementor['escolheu_profissao']
df_leads_elementor['escolheu_profissao'] = df_leads_elementor['escolheu_profissao'].apply(normalizar_escolheu_profissao)

# -------------------------------------
# Filtrar registros com valores válidos
# -------------------------------------
df_leads_elementor = df_leads_elementor[df_leads_elementor['escolheu_profissao'].isin(MOTIVOS_VALIDOS)]

# -------------------------------------
# Ver resultado final
# -------------------------------------
with pd.option_context('display.max_rows', None):
    print(df_leads_elementor['escolheu_profissao'].value_counts(dropna=False))

escolheu_profissao
sonho de criança           5010
gosta da profissão         4966
estabilidade de emprego    2918
prestígio da carreira      1289
Name: count, dtype: int64


In [42]:
# -------------------------------------
# Categorias simples padronizadas
# -------------------------------------
DIFICULDADES_VALIDAS = {
    'financeiro / dinheiro',
    'falta de tempo',
    'falta de base escolar',
    'falta de oportunidade',
    'idade'
}

# -------------------------------------
# Mapeamento de variações conhecidas
# -------------------------------------
SUBSTITUICOES = {
    'falta de oportunidades na vida': 'falta de oportunidade',
    'financeiro \\/ dinheiro': 'financeiro / dinheiro'
}

# -------------------------------------
# Função de limpeza
# -------------------------------------
def normalizar_dificuldade(valor):
    if pd.isna(valor):
        return None
    val = str(valor).strip().lower()

    for k, v in SUBSTITUICOES.items():
        val = val.replace(k, v)

    if ',' in val:
        return None

    return val if val in DIFICULDADES_VALIDAS else None

# -------------------------------------
# Aplicar ao DataFrame
# -------------------------------------
df_leads_elementor['dificuldade_original'] = df_leads_elementor['dificuldade']
df_leads_elementor['dificuldade'] = df_leads_elementor['dificuldade'].apply(normalizar_dificuldade)

# -------------------------------------
# Filtrar registros com dificuldades válidas
# -------------------------------------
df_leads_elementor = df_leads_elementor[df_leads_elementor['dificuldade'].isin(DIFICULDADES_VALIDAS)]

# -------------------------------------
# Ver resultado final
# -------------------------------------
with pd.option_context('display.max_rows', None):
    print(df_leads_elementor['dificuldade'].value_counts(dropna=False))

dificuldade
financeiro / dinheiro    4586
falta de tempo           2764
falta de base escolar    1789
idade                    1186
falta de oportunidade    1104
Name: count, dtype: int64


In [43]:
df_leads_elementor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11429 entries, 0 to 24348
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   data                         11429 non-null  datetime64[ns]
 1   lancamentos                  11429 non-null  object        
 2   email                        11429 non-null  object        
 3   whatsapp                     10390 non-null  object        
 4   estado                       11240 non-null  object        
 5   idade                        10993 non-null  object        
 6   escolaridade                 10714 non-null  object        
 7   renda                        11429 non-null  object        
 8   estado_civil                 11429 non-null  object        
 9   filhos                       11429 non-null  object        
 10  escolheu_profissao           11429 non-null  object        
 11  dificuldade                  11429 non-null  o

In [44]:
df_leads_elementor.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade,renda_original,estado_civil_original,filhos_original,escolheu_profissao_original,dificuldade_original
0,2025-04-08 16:04:31,L33,MA,36 - 45 anos,superior completo,de 3.000 a 5.000,união estável,sim,prestígio da carreira,falta de tempo,De 3.000 a 5.000,união estável,Sim,Prestígio da carreira,Falta de tempo
5,2025-04-05 00:00:22,L33,SP,26 - 35 anos,superior completo,desempregado,casado(a),não,estabilidade de emprego,falta de tempo,Desempregado,casado(a),Não,Estabilidade de emprego,Falta de tempo
10,2025-04-04 00:46:06,L33,SP,Até 25 anos,superior completo,até 1.000,solteiro(a),não,sonho de criança,falta de base escolar,Até 1.000,solteiro(a),Não,Sonho de criança,Falta de base escolar
11,2025-04-03 21:59:07,L33,SP,26 - 35 anos,médio completo,de 1.000 a 3.000,união estável,não,gosta da profissão,financeiro / dinheiro,De 1.000 a 3.000,união estável,Não,Gosta da profissão,Financeiro \/ Dinheiro
18,2025-04-02 21:20:15,L33,SP,46 - 55 anos,médio completo,de 1.000 a 3.000,solteiro(a),não,sonho de criança,falta de tempo,De 1.000 a 3.000,solteiro(a),Não,Sonho de criança,Falta de tempo


In [45]:
# Excluir colunas que terminam com "_original"
df_leads_elementor.drop(
    columns=[col for col in df_leads_elementor.columns if col.endswith('_original')],
    inplace=True
)

In [46]:
df_leads_elementor.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11429 entries, 0 to 24348
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                11429 non-null  datetime64[ns]
 1   lancamentos         11429 non-null  object        
 2   email               11429 non-null  object        
 3   whatsapp            10390 non-null  object        
 4   estado              11240 non-null  object        
 5   idade               10993 non-null  object        
 6   escolaridade        10714 non-null  object        
 7   renda               11429 non-null  object        
 8   estado_civil        11429 non-null  object        
 9   filhos              11429 non-null  object        
 10  escolheu_profissao  11429 non-null  object        
 11  dificuldade         11429 non-null  object        
dtypes: datetime64[ns](1), object(11)
memory usage: 1.1+ MB


In [47]:
print("Google:", len(df_leads_google))
print("Elementor:", len(df_leads_elementor))
print("Antigos:", len(df_leads_antigos))

Google: 1955
Elementor: 11429
Antigos: 1553


# Merge `df_google` | `df_elementor` | `df_leads_antigos`

In [48]:
# Conjuntos de e-mails
emails_google = set(df_leads_google['email'].unique())
emails_elementor = set(df_leads_elementor['email'].unique())
emails_leads_antigos = set(df_leads_antigos['email'].unique())

# Interseção: e-mails que aparecem nos dois
emails_em_ambos = emails_google & emails_elementor & emails_leads_antigos

print(f"E-mails duplicados: {len(emails_em_ambos)}")

E-mails duplicados: 6


In [49]:
# Recarregue os dois DataFrames originais se possível (ou copie)
df_leads_google = df_leads_google.copy()
df_leads_elementor = df_leads_elementor.copy()
df_leads_antigos = df_leads_antigos.copy()

# Junte corretamente
df_leads = pd.concat([df_leads_google, df_leads_elementor, df_leads_antigos], ignore_index=True)
print("Total:", len(df_leads))

Total: 14937


In [50]:
df_leads = df_leads.drop_duplicates(subset=['email', 'lancamentos'])
print("Total:", len(df_leads))

Total: 14183


In [51]:
# Colunas críticas que devem estar completas
colunas_obrigatorias = [
    'estado',
    'idade',
    'escolaridade',
    'renda',
    'estado_civil',
    'filhos',
    'escolheu_profissao',
    'dificuldade'
]

# Remove registros com qualquer valor ausente nas colunas-chave
df_leads = df_leads.dropna(subset=colunas_obrigatorias)

# Exibir total de registros após limpeza
print("Total após limpeza completa:", len(df_leads))

Total após limpeza completa: 12980


In [52]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12980 entries, 0 to 14936
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                12980 non-null  object
 1   lancamentos         12980 non-null  object
 2   email               12980 non-null  object
 3   whatsapp            12140 non-null  object
 4   estado              12980 non-null  object
 5   idade               12980 non-null  object
 6   escolaridade        12980 non-null  object
 7   renda               12980 non-null  object
 8   estado_civil        12980 non-null  object
 9   filhos              12980 non-null  object
 10  escolheu_profissao  12980 non-null  object
 11  dificuldade         12980 non-null  object
 12  nome                1517 non-null   object
dtypes: object(13)
memory usage: 1.4+ MB


In [53]:
df_leads["comprou"] = df_leads["email"].isin(df_compradores["email"]).astype(int)

In [54]:
print("Duplicados por email:", df_leads['email'].duplicated().sum())

Duplicados por email: 228


In [55]:
df_leads.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade,nome,comprou
0,2025-03-17 10:41:52,L33,RJ,Acima de 56 anos,fundamental completo,Até 1.000,casado(a),Sim,gosta da profissão,idade,NaN,0
1,2025-03-17 10:42:16,L33,SP,Acima de 56 anos,médio completo,De 3.000 a 5.000,casado(a),Sim,gosta da profissão,falta de base escolar,NaN,0
2,2025-03-17 10:44:49,L33,MG,36 - 45 anos,médio completo,Até 1.000,solteiro(a),Sim,estabilidade de emprego,financeiro / dinheiro,NaN,0
3,2025-03-17 10:50:45,L33,SP,36 - 45 anos,médio completo,De 1.000 a 3.000,casado(a),Sim,gosta da profissão,financeiro / dinheiro,NaN,1
4,2025-03-17 11:07:02,L33,SP,26 - 35 anos,fundamental completo,De 1.000 a 3.000,solteiro(a),Sim,gosta da profissão,falta de base escolar,NaN,0


In [56]:
df_compradores_responderam = df_leads[
    df_leads["email"].isin(df_compradores["email"])
].copy()

In [57]:
print(f"{df_compradores_responderam.shape[0]} alunos encontrados no df_leads.")

307 alunos encontrados no df_leads.


In [58]:
df_compradores_responderam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307 entries, 3 to 14933
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                307 non-null    object
 1   lancamentos         307 non-null    object
 2   email               307 non-null    object
 3   whatsapp            302 non-null    object
 4   estado              307 non-null    object
 5   idade               307 non-null    object
 6   escolaridade        307 non-null    object
 7   renda               307 non-null    object
 8   estado_civil        307 non-null    object
 9   filhos              307 non-null    object
 10  escolheu_profissao  307 non-null    object
 11  dificuldade         307 non-null    object
 12  nome                78 non-null     object
 13  comprou             307 non-null    int32 
dtypes: int32(1), object(13)
memory usage: 34.8+ KB


In [59]:
df_alunos_nao_respondentes = df_compradores[
    ~df_compradores["email"].isin(df_leads["email"])
].copy()

In [60]:
df_alunos_nao_respondentes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1199 entries, 0 to 1484
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   nome             637 non-null    object        
 1   email            1199 non-null   object        
 2   papel            637 non-null    object        
 3   acesso           637 non-null    object        
 4   turma            637 non-null    object        
 5   categoria        637 non-null    object        
 6   primeiro_acesso  604 non-null    datetime64[ns]
 7   ultimo_acesso    604 non-null    datetime64[ns]
 8   progresso        637 non-null    float64       
 9   engajamento      637 non-null    object        
 10  data             637 non-null    datetime64[ns]
 11  numero_acesso    637 non-null    float64       
 12  lançamento       562 non-null    object        
dtypes: datetime64[ns](3), float64(2), object(8)
memory usage: 131.1+ KB


In [61]:
df_alunos = df_compradores_responderam.copy()

# Opcional: garantir que as colunas estão na mesma ordem
df_alunos = df_alunos[df_leads.columns]

df_alunos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 307 entries, 3 to 14933
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                307 non-null    object
 1   lancamentos         307 non-null    object
 2   email               307 non-null    object
 3   whatsapp            302 non-null    object
 4   estado              307 non-null    object
 5   idade               307 non-null    object
 6   escolaridade        307 non-null    object
 7   renda               307 non-null    object
 8   estado_civil        307 non-null    object
 9   filhos              307 non-null    object
 10  escolheu_profissao  307 non-null    object
 11  dificuldade         307 non-null    object
 12  nome                78 non-null     object
 13  comprou             307 non-null    int32 
dtypes: int32(1), object(13)
memory usage: 34.8+ KB


In [62]:
features = ["renda", "escolaridade", "idade", "estado_civil", "filhos", 'escolaridade', 'escolheu_profissao', 'dificuldade']

for col in features:
    if col in df_leads.columns:
        df_leads[col] = df_leads[col].astype(str).str.strip().str.lower()

for col in features:
    if col in df_alunos.columns:
        df_alunos[col] = df_alunos[col].astype(str).str.strip().str.lower()

In [63]:
df_alunos.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade,nome,comprou
3,2025-03-17 10:50:45,L33,SP,36 - 45 anos,médio completo,de 1.000 a 3.000,casado(a),sim,gosta da profissão,financeiro / dinheiro,NaN,1
24,2025-03-17 20:40:51,L33,SP,46 - 55 anos,médio completo,de 1.000 a 3.000,união estável,sim,gosta da profissão,financeiro / dinheiro,NaN,1
32,2025-03-17 22:23:52,L33,SP,26 - 35 anos,médio completo,de 3.000 a 5.000,solteiro(a),não,sonho de criança,financeiro / dinheiro,NaN,1
45,2025-03-18 11:53:16,L33,SP,46 - 55 anos,superior completo,acima de 5.000,casado(a),sim,sonho de criança,falta de tempo,NaN,1
80,2025-03-19 07:46:45,L33,RJ,36 - 45 anos,superior completo,de 3.000 a 5.000,casado(a),sim,sonho de criança,falta de tempo,NaN,1


In [64]:
colunas_excluir = ["data", "email", "whatsapp", "nome"]

for coluna in df_leads.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L32    5770
L31    2973
L33    2720
L30     594
L29     487
L28     436
Name: count, dtype: int64

Coluna: estado
estado
SP       9600
RJ       1741
MG       1232
Outro     248
PR         54
MA         48
BA         15
GO         12
SC          5
RS          5
DF          4
AL          3
CE          3
RR          2
PE          2
PA          2
AC          1
RO          1
AM          1
RN          1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        4968
26 - 35 anos        4047
46 - 55 anos        2806
até 25 anos          947
acima de 56 anos     212
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            9466
superior completo         2214
fundamental completo      1096
superior incompleto        124
técnico                     34
médio incompleto            24
fundamental incompleto      22
Name: count, dtype: int64

Coluna: renda
renda
de 1.000 a 3.000    6851
de 3.000 a 5.000    2985
até 1.000          

In [65]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_alunos.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_alunos[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L32    97
L31    82
L33    50
L30    27
L29    26
L28    25
Name: count, dtype: int64

Coluna: estado
estado
SP       266
RJ        19
MG        15
Outro      3
DF         1
PR         1
SC         1
RS         1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        117
26 - 35 anos        101
46 - 55 anos         63
até 25 anos          23
acima de 56 anos      3
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo          219
superior completo        62
fundamental completo     15
técnico                   5
superior incompleto       5
médio incompleto          1
Name: count, dtype: int64

Coluna: renda
renda
de 1.000 a 3.000    169
de 3.000 a 5.000     72
desempregado         28
acima de 5.000       22
até 1.000            16
Name: count, dtype: int64

Coluna: estado_civil
estado_civil
casado(a)                       154
solteiro(a)                     103
união estável                    32
divorciado(a) ou separ

# Adicionar UTM's

In [66]:
df_utms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59841 entries, 0 to 59840
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   email              59841 non-null  object 
 1   url                59841 non-null  object 
 2   utm_campaign       59752 non-null  object 
 3   utm_source         59760 non-null  object 
 4   utm_medium         59149 non-null  object 
 5   utm_content        55478 non-null  object 
 6   utm_term           37063 non-null  object 
 7   ltm_campaign_id    17562 non-null  float64
 8   ltm_campaign_name  17562 non-null  object 
 9   ltm_adgroup_id     17562 non-null  float64
 10  ltm_adgroup_name   17562 non-null  object 
 11  ltm_ad_id          17562 non-null  float64
 12  ltm_ad_name        15621 non-null  object 
 13  ltm_network        17562 non-null  object 
 14  ltm_type           17562 non-null  object 
 15  date               59841 non-null  object 
dtypes: float64(3), object(

In [67]:
df_utms_antigas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34685 entries, 0 to 34684
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   email              34685 non-null  object 
 1   url                34685 non-null  object 
 2   utm_campaign       34338 non-null  object 
 3   utm_source         34393 non-null  object 
 4   utm_medium         34338 non-null  object 
 5   utm_content        33096 non-null  object 
 6   utm_term           19514 non-null  object 
 7   ltm_campaign_id    12258 non-null  float64
 8   ltm_campaign_name  12258 non-null  object 
 9   ltm_adgroup_id     12258 non-null  object 
 10  ltm_adgroup_name   12258 non-null  object 
 11  ltm_ad_id          12258 non-null  object 
 12  ltm_ad_name        11347 non-null  object 
 13  ltm_network        12258 non-null  object 
 14  ltm_type           12258 non-null  object 
 15  date               34685 non-null  object 
dtypes: float64(1), object(

In [68]:
df_utms = pd.concat([df_utms, df_utms_antigas], ignore_index=True)

In [69]:
df_utms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94526 entries, 0 to 94525
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   email              94526 non-null  object 
 1   url                94526 non-null  object 
 2   utm_campaign       94090 non-null  object 
 3   utm_source         94153 non-null  object 
 4   utm_medium         93487 non-null  object 
 5   utm_content        88574 non-null  object 
 6   utm_term           56577 non-null  object 
 7   ltm_campaign_id    29820 non-null  float64
 8   ltm_campaign_name  29820 non-null  object 
 9   ltm_adgroup_id     29820 non-null  object 
 10  ltm_adgroup_name   29820 non-null  object 
 11  ltm_ad_id          29820 non-null  object 
 12  ltm_ad_name        26968 non-null  object 
 13  ltm_network        29820 non-null  object 
 14  ltm_type           29820 non-null  object 
 15  date               94526 non-null  object 
dtypes: float64(1), object(

In [70]:
# Colunas que queremos manter
colunas_utms = [
    'email',
    'url',
    'utm_source',
    'utm_medium',
    'utm_campaign',
    'utm_content',
    'utm_term',
    'date'
]

# Filtrar o DataFrame
df_utms = df_utms[colunas_utms]

In [71]:
df_utms.drop(columns=['email']).head()

,url,utm_source,utm_medium,utm_campaign,utm_content,utm_term,date
0,missaopcsp.qgconcursos.com/l031-missao-pcsp-me...,Instagram_Feed,04_concorrentes,L31,Video_vemai_v1,frio,19/11/2024 - 00h00m14s
1,missaopcsp.qgconcursos.com/l031-missao-pcsp-org-d,BIO,NaN,Instagram,NaN,NaN,19/11/2024 - 00h02m40s
2,missaopcsp.qgconcursos.com/l031-missao-pcsp-me...,Instagram_Reels,04_ll_1_envolvimento_30d,L31,Video_vemai_v1,frio,19/11/2024 - 00h03m50s
3,missaopcsp.qgconcursos.com/l031-missao-pcsp-me...,Instagram_Feed,00_envolvimento_60d_seguidores,L31,vÃ­deo_01,quente,19/11/2024 - 00h05m13s
4,missaopcsp.qgconcursos.com/l031-missao-pcsp-me...,Instagram_Reels,04_ll_1_envolvimento_30d,L31,Video_vemai_v1,frio,19/11/2024 - 00h06m06s


In [72]:
# Padronizar nomes das colunas
df_utms.columns = df_utms.columns.str.strip().str.lower().str.replace(" ", "_")

# Definir colunas UTM
colunas_utms = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content', 'utm_term']

# Remover linhas com qualquer "teste" (em qualquer coluna)
df_utms = df_utms[
    ~df_utms.apply(lambda row: row.astype(str).str.lower().str.contains('teste').any(), axis=1)
]

# Resetar índice
df_utms.reset_index(drop=True, inplace=True)

# Ver resultado
df_utms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94489 entries, 0 to 94488
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   email         94489 non-null  object
 1   url           94489 non-null  object
 2   utm_source    94136 non-null  object
 3   utm_medium    93471 non-null  object
 4   utm_campaign  94074 non-null  object
 5   utm_content   88559 non-null  object
 6   utm_term      56569 non-null  object
 7   date          94489 non-null  object
dtypes: object(8)
memory usage: 5.8+ MB


In [73]:
# 1. Garantir datetime correto
df_leads['data'] = pd.to_datetime(df_leads['data'])
df_utms['date'] = pd.to_datetime(df_utms['date'], dayfirst=True)

# 2. Merge por email (isso cria múltiplas linhas por lead se ele clicou várias vezes)
df_leads_temp = df_leads.merge(df_utms, on='email', how='left')

# 3. Filtrar apenas UTMs que aconteceram antes ou no momento da entrada do lead
df_leads_temp = df_leads_temp[df_leads_temp['date'] <= df_leads_temp['data']]

# 4. Ordenar para pegar o last-touch (última interação antes da entrada)
df_leads_temp = df_leads_temp.sort_values(by=['email', 'lancamentos', 'date'], ascending=[True, True, False])

# 5. Drop para manter só uma linha por email + lançamento (a correta)
df_leads = df_leads_temp.drop_duplicates(subset=['email', 'lancamentos'], keep='first').reset_index(drop=True)

In [74]:
# Contar quantos lançamentos únicos cada lead participou
lead_lanc_count = df_leads.groupby('email')['lancamentos'].nunique()

# Filtrar apenas os leads que participaram de MAIS DE UM lançamento
leads_multiplos_lanc = lead_lanc_count[lead_lanc_count > 1]

# Mostrar alguns exemplos
df_leads[df_leads['email'].isin(leads_multiplos_lanc.index)].sort_values(['email', 'data']).head(10)


,data,lancamentos,email,whatsapp,estado,idade,escolaridade,renda,estado_civil,filhos,...,dificuldade,nome,comprou,url,utm_source,utm_medium,utm_campaign,utm_content,utm_term,date
15,2024-11-27 16:31:47,L31,9163ferraz@gmail.com,14991933798,SP,36 - 45 anos,médio completo,de 1.000 a 3.000,casado(a),sim,...,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l031-missao-pcsp-go...,tva,00_Envolvimento_yt_180d,L31,Video_entrevista_v2_yt,NaN,2024-11-27 16:15:24
16,2025-01-21 12:49:59,L32,9163ferraz@gmail.com,14991933798,SP,36 - 45 anos,médio completo,até 1.000,casado(a),sim,...,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l32-operacao-polici...,tva,00_listas,L32,Video_vemai_v1_yt,NaN,2025-01-21 11:50:03
94,2024-05-25 13:07:00,L28,adilsonbarbosadeoliveira@gmail.com,11989602090,SP,36 - 45 anos,médio completo,de 1.000 a 3.000,casado(a),sim,...,financeiro / dinheiro,Adilson Barbosa de Oliveira,0,qgconcursos.com/missaopcsp2024-l28-ga,tva,00_listas,L28,Video_5_motivos_st_yt,NaN,2024-05-23 06:26:05
95,2024-11-23 10:53:04,L31,adilsonbarbosadeoliveira@gmail.com,11989602090,SP,36 - 45 anos,médio completo,de 1.000 a 3.000,casado(a),sim,...,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l031-missao-pcsp-go...,tva,00_Envolvimento_yt_180d,L31,Video_caminhoneiro_yt,NaN,2024-11-22 12:26:06
106,2025-01-20 12:12:45,L32,adm.jvcardoso@gmail.com,18997685615,SP,36 - 45 anos,superior completo,de 1.000 a 3.000,casado(a),sim,...,falta de tempo,NaN,0,missaopcsp.qgconcursos.com/l32-operacao-polici...,Instagram_Stories,04_concorrentes,L32,imagem_sobral,frio,2025-01-20 11:16:01
107,2025-03-31 17:13:31,L33,adm.jvcardoso@gmail.com,18997685615,SP,36 - 45 anos,superior completo,de 1.000 a 3.000,casado(a),sim,...,falta de tempo,NaN,0,missaopcsp.qgconcursos.com/l33-operacao-polici...,Instagram_Feed,00_listas_ultimos_lctos,L33,video_mais_um_concurso,quente,2025-03-31 17:08:01
116,2024-11-25 12:15:24,L31,adriana_olher@outlook.com,18996450638,SP,26 - 35 anos,médio completo,de 1.000 a 3.000,casado(a),sim,...,falta de tempo,NaN,0,missaopcsp.qgconcursos.com/l031-missao-pcsp-go...,tva,00_Envolvimento_yt_180d,L31,Video_entrevista_v2_yt,NaN,2024-11-25 11:57:57
117,2025-02-04 01:48:49,L32,adriana_olher@outlook.com,18996450638,SP,26 - 35 anos,fundamental completo,de 3.000 a 5.000,casado(a),sim,...,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l32-operacao-polici...,tva,00_envolvimento_yt_180d,L32,Video_vespera_do_concurso_yt,NaN,2025-02-04 00:51:00
154,2024-07-25 18:58:00,L29,adrianomachado048@gmail.com,15997375078,SP,46 - 55 anos,superior completo,de 1.000 a 3.000,solteiro(a),não,...,financeiro / dinheiro,Adriano da Silva Machado,0,qgconcursos.com/l29-missao-pcsp-nivel-medio-go...,tva,00_listas,L29,Video_obj_cartao_yt,NaN,2024-07-24 23:45:27
155,2024-12-04 05:13:23,L31,adrianomachado048@gmail.com,15997375078,SP,46 - 55 anos,superior completo,de 1.000 a 3.000,solteiro(a),não,...,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l31-missao-pcsp-met...,Facebook_Mobile_Feed,00_listas_ultimos_lctos,L31,video_salario_7k,quente,2024-12-04 05:11:13


In [75]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10863 entries, 0 to 10862
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                10863 non-null  datetime64[ns]
 1   lancamentos         10863 non-null  object        
 2   email               10863 non-null  object        
 3   whatsapp            10127 non-null  object        
 4   estado              10863 non-null  object        
 5   idade               10863 non-null  object        
 6   escolaridade        10863 non-null  object        
 7   renda               10863 non-null  object        
 8   estado_civil        10863 non-null  object        
 9   filhos              10863 non-null  object        
 10  escolheu_profissao  10863 non-null  object        
 11  dificuldade         10863 non-null  object        
 12  nome                870 non-null    object        
 13  comprou             10863 non-null  int32     

In [76]:
# 1. Garantir datetime correto
df_alunos['data'] = pd.to_datetime(df_alunos['data'])
df_utms['date'] = pd.to_datetime(df_utms['date'], dayfirst=True)

# 2. Merge por email (isso cria múltiplas linhas por lead se ele clicou várias vezes)
df_alunos_temp = df_alunos.merge(df_utms, on='email', how='left')

# 3. Filtrar apenas UTMs que aconteceram antes ou no momento da entrada do lead
df_alunos_temp = df_alunos_temp[df_alunos_temp['date'] <= df_alunos_temp['data']]

# 4. Ordenar para pegar o last-touch (última interação antes da entrada)
df_alunos_temp = df_alunos_temp.sort_values(by=['email', 'lancamentos', 'date'], ascending=[True, True, False])

# 5. Drop para manter só uma linha por email + lançamento (a correta)
df_alunos = df_alunos_temp.drop_duplicates(subset=['email', 'lancamentos'], keep='first').reset_index(drop=True)

In [77]:
df_alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                260 non-null    datetime64[ns]
 1   lancamentos         260 non-null    object        
 2   email               260 non-null    object        
 3   whatsapp            255 non-null    object        
 4   estado              260 non-null    object        
 5   idade               260 non-null    object        
 6   escolaridade        260 non-null    object        
 7   renda               260 non-null    object        
 8   estado_civil        260 non-null    object        
 9   filhos              260 non-null    object        
 10  escolheu_profissao  260 non-null    object        
 11  dificuldade         260 non-null    object        
 12  nome                54 non-null     object        
 13  comprou             260 non-null    int32         

In [78]:
def limpar_utm(valor):
    if pd.isna(valor):
        return np.nan
    valor = str(valor).strip().lower()

    # Remove placeholders do tipo {{...}}
    if re.match(r"\{\{.*\}\}", valor):
        return np.nan

    # Remove lixo de codificação (acentuação quebrada tipo 'descriã§ã£o')
    valor = unicodedata.normalize("NFKD", valor).encode("ASCII", "ignore").decode("utf-8")

    # Substitui valores claramente inválidos
    if valor in ["", "nan", "none", "n/a", "-"]:
        return np.nan

    return valor

def tratar_utms(df):
    for campo in colunas_utms:
        if campo in df.columns:
            df[campo] = df[campo].apply(limpar_utm)
    return df

In [79]:
df_leads = tratar_utms(df_leads)
df_alunos = tratar_utms(df_alunos)

In [80]:
df_leads.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade,nome,comprou,url,utm_source,utm_medium,utm_campaign,utm_content,utm_term,date
0,2025-01-27 17:31:57,L32,SP,26 - 35 anos,médio completo,de 3.000 a 5.000,união estável,sim,sonho de criança,falta de base escolar,NaN,0,missaopcsp.qgconcursos.com/l32-operacao-polici...,whatsapp,grupos antigos,l32,NaN,NaN,2025-01-27 16:44:00
1,2024-11-20 10:56:47,L31,SP,46 - 55 anos,médio completo,até 1.000,divorciado(a) ou separado(a),sim,gosta da profissão,falta de oportunidade,NaN,0,missaopcsp.qgconcursos.com/l031-missao-pcsp-me...,instagram_feed,04_ll_1_envolvimento_30d,l31,video_vemai_v1,frio,2024-11-20 10:42:44
2,2025-01-28 01:45:29,L32,SP,26 - 35 anos,fundamental completo,desempregado,solteiro(a),não,sonho de criança,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l32-operacao-polici...,tva,04_afinidade_luta,l32,video_5_motivos_yt,NaN,2025-01-28 00:26:23
3,2025-03-17 20:27:57,L33,SP,26 - 35 anos,médio completo,desempregado,solteiro(a),sim,sonho de criança,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l33-operacao-polici...,instagram,bio,l33,NaN,NaN,2025-03-17 19:38:51
4,2025-03-23 22:39:51,L33,SP,26 - 35 anos,médio completo,de 1.000 a 3.000,solteiro(a),sim,gosta da profissão,financeiro / dinheiro,NaN,0,missaopcsp.qgconcursos.com/l33-operacao-polici...,instagram_stories,04_ll_10_lista_de_alunos,l33,video_vemai_v1_pag_f,frio,2025-03-23 21:34:35


In [81]:
df_alunos.drop(columns=['email', 'whatsapp']).head()

,data,lancamentos,estado,idade,escolaridade,renda,estado_civil,filhos,escolheu_profissao,dificuldade,nome,comprou,url,utm_source,utm_medium,utm_campaign,utm_content,utm_term,date
0,2024-07-29 13:51:00,L29,SP,46 - 55 anos,médio completo,de 3.000 a 5.000,casado(a),sim,gosta da profissão,financeiro / dinheiro,Alexandre Menecate,1,qgconcursos.com/l29-missao-pcsp-nivel-medio-or...,youtube,comunidade,l29,posts,NaN,2024-07-28 02:35:15
1,2024-09-20 11:44:00,L30,SP,46 - 55 anos,médio completo,de 3.000 a 5.000,casado(a),sim,sonho de criança,financeiro / dinheiro,Adailton da Silva Almeida,1,qgconcursos.com/l30-missao-pcsp-nivel-medio-go...,tva,00_envolvimento_yt_180d,l30,video_atras_do_sonho_v1_yt,NaN,2024-09-16 22:06:56
2,2025-03-21 11:38:46,L33,SP,46 - 55 anos,superior completo,desempregado,casado(a),não,gosta da profissão,idade,NaN,1,missaopcsp.qgconcursos.com/l33-operacao-polici...,tva,00_envolvimento_yt_180d,l33,video_caminhoneiro_yt,NaN,2025-03-20 03:12:03
3,2025-01-28 18:52:59,L32,SP,até 25 anos,médio completo,de 1.000 a 3.000,casado(a),não,sonho de criança,idade,NaN,1,missaopcsp.qgconcursos.com/l32-operacao-polici...,instagram,manychat,l32,NaN,NaN,2025-01-28 15:19:54
4,2025-01-20 08:28:02,L32,RJ,36 - 45 anos,superior completo,de 1.000 a 3.000,casado(a),não,gosta da profissão,falta de tempo,NaN,1,missaopcsp.qgconcursos.com/l32-operacao-polici...,tva,04_mercado_concurso,l32,video_entrevista_v2_yt,NaN,2025-01-18 07:57:29


In [82]:
df_leads['whatsapp'] = df_leads['whatsapp'].astype(str)
df_leads = df_leads.drop(columns=['date', 'url'])

df_alunos['whatsapp'] = df_alunos['whatsapp'].astype(str)
df_alunos = df_alunos.drop(columns=['date', 'url'])

In [83]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10863 entries, 0 to 10862
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                10863 non-null  datetime64[ns]
 1   lancamentos         10863 non-null  object        
 2   email               10863 non-null  object        
 3   whatsapp            10863 non-null  object        
 4   estado              10863 non-null  object        
 5   idade               10863 non-null  object        
 6   escolaridade        10863 non-null  object        
 7   renda               10863 non-null  object        
 8   estado_civil        10863 non-null  object        
 9   filhos              10863 non-null  object        
 10  escolheu_profissao  10863 non-null  object        
 11  dificuldade         10863 non-null  object        
 12  nome                870 non-null    object        
 13  comprou             10863 non-null  int32     

In [84]:
df_alunos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   data                260 non-null    datetime64[ns]
 1   lancamentos         260 non-null    object        
 2   email               260 non-null    object        
 3   whatsapp            260 non-null    object        
 4   estado              260 non-null    object        
 5   idade               260 non-null    object        
 6   escolaridade        260 non-null    object        
 7   renda               260 non-null    object        
 8   estado_civil        260 non-null    object        
 9   filhos              260 non-null    object        
 10  escolheu_profissao  260 non-null    object        
 11  dificuldade         260 non-null    object        
 12  nome                54 non-null     object        
 13  comprou             260 non-null    int32         

# Tratamento de Dados - Leads Novos

In [85]:
# === Carregar variáveis de ambiente ===
load_dotenv(Path.cwd().parent / "secrets" / ".env", override=True)

# === Definir escopos de acesso ===
scopes = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly"
]

# === Caminho seguro para o arquivo JSON (lido do .env) ===
cred_path = Path.cwd().parent / os.getenv("GOOGLE_CREDENTIALS_PATH")

# === Carregar credenciais de forma moderna ===
creds = service_account.Credentials.from_service_account_file(
    str(cred_path),
    scopes=scopes
)

# === Autorizar o cliente gspread ===
client = gspread.authorize(creds)

# === Função para carregar aba da planilha ===
def carregar_aba(sheet_id, aba_nome):
    planilha = client.open_by_key(sheet_id)
    aba = planilha.worksheet(aba_nome)
    dados = aba.get_all_records()
    return pd.DataFrame(dados)

# === IDs das planilhas (fixar no .env depois se quiser) ===
id_leads_l34 = "155-8YH18_ExLOLq0kWQTz1rolB4903hV6_MfD1FubkQ"

# === Carregar DataFrames ===
df_leads_l34_forms = carregar_aba(id_leads_l34, "Respostas ao formulário 1")

df_leads_l34_forms.head()

,Carimbo de data/hora,Nome completo,Seu melhor e-mail,Número de WhatsApp,Qual estado você reside?,Qual sua faixa de idade?,Qual seu nível de escolaridade?,Qual sua faixa salarial atualmente?,Qual seu estado civil?,Você tem filhos?,Por que você escolheu/escolheria a profissão de Policial Civil?,O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?,O que você espera aprender/ver no evento Operação Policial Civil - 2025?,"Imagine que você está em uma mentoria individual com o Prof. Luis Costa, quais seriam as 3 principais perguntas que você faria para ele?"
0,16/05/2025 22:06:54,Camilo Bica Fonseca,camilobf2@gmail.com,73991808783,Minas Gerais,Até 25 anos,Ensino Fundamental Completo,"Até R$ 1.000,00",Solteiro(a),Sim,Sonho de criança,Falta de base escolar,Como ser policial,teste
1,17/05/2025 15:16:14,Rodrigo de Souza silva,rodrigo238712@gmail.com,49 99807-6654,Bahia ilha de Pauloafonso,36 - 45 anos,Ensino Médio Completo,"De R$ 3.000,00 a R$ 5.000,00",Solteiro(a),Sim,Sonho de criança,Falta de tempo,"Como ser policial, Adquirir conhecimento, Cond...",Oque preciso para ser um policial civil. Oque ...
2,17/05/2025 15:36:36,Fernando Fernandes Sapucaia,sapucaiafer75@gmail.com,13997141460,São Paulo,36 - 45 anos,Ensino Superior Completo,"De R$ 1.000,00 a R$ 3.000,00",Solteiro(a),Não,Gosta da profissão,Financeiro / Dinheiro,Concurso (base/material),Plano de carreira?\nPorque se tornou professor...
3,18/05/2025 00:59:05,David Roberto Duarte Alves,daresaem@gmail.com,17981326552,São Paulo,46 - 55 anos,Ensino Superior Completo,"De R$ 3.000,00 a R$ 5.000,00",Casado(a),Sim,Sonho de criança,Falta de tempo,"Adquirir conhecimento, Como estudar",Qual a maior dificuldade para passar na prova ...
4,18/05/2025 09:07:21,Márcia Regina serafim,marcialeserafim@gmail.com,17992389678,São Paulo,36 - 45 anos,Ensino Superior Completo,"De R$ 1.000,00 a R$ 3.000,00",Solteiro(a),Sim,Sonho de criança,Falta de tempo,Como ser aprovado,Nós teremos apostila.tenho um pouco de dificul...


In [86]:
df_leads_l34_forms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 14 columns):
 #   Column                                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                                    --------------  ----- 
 0   Carimbo de data/hora                                                                                                                      478 non-null    object
 1   Nome completo                                                                                                                             478 non-null    object
 2   Seu melhor e-mail                                                                                                                         478 non-null    object
 3   Número de WhatsApp                                          

In [87]:
df_leads_l34_forms.drop(columns=[
    "O que você espera aprender/ver no evento Operação Policial Civil - 2025? ",
    "Imagine que você está em uma mentoria individual com o Prof. Luis Costa, quais seriam as 3 principais perguntas que você faria para ele?"
], inplace=True)

In [88]:
# Renomear colunas do df_leads_google
colunas_renomeadas = {
    'Carimbo de data/hora': 'data',
    'Seu melhor e-mail': 'email',
    'Nome completo': 'nome',
    'Número de WhatsApp': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'Qual seu estado civil?': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'Por que você escolheu/escolheria a profissão de Policial Civil?': 'escolheu_profissao',
    'O que está IMPEDINDO você de conquistar o seu DISTINTIVO e ser o próximo POLICIAL CIVIL? Qual sua maior dificuldade?': 'dificuldade'
}

df_leads_l34_forms.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_l34_forms.columns:
    df_leads_l34_forms['email'] = df_leads_l34_forms['email'].astype(str).str.lower().str.strip()

# Exibir resumo
df_leads_l34_forms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                478 non-null    object
 1   nome                478 non-null    object
 2   email               478 non-null    object
 3   whatsapp            478 non-null    object
 4   estado              478 non-null    object
 5   idade               478 non-null    object
 6   escolaridade        478 non-null    object
 7   renda               478 non-null    object
 8   estado_civil        478 non-null    object
 9   filhos              478 non-null    object
 10  escolheu_profissao  478 non-null    object
 11  dificuldade         478 non-null    object
dtypes: object(12)
memory usage: 44.9+ KB


In [89]:
df_leads_l34_forms["renda"] = (
    df_leads_l34_forms["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [90]:
# Aplicar a função
df_leads_l34_forms['estado'] = df_leads_l34_forms['estado'].apply(normalizar_estado)

# Verificar resultado final
print(df_leads_l34_forms['estado'].value_counts(dropna=False))

estado
SP                 294
MG                  45
BA                  29
PE                  19
RS                  18
PR                  14
RJ                  13
SC                  12
AL                  11
MA                   5
ES                   3
CE                   3
GO                   3
PA                   2
RN                   2
TO                   1
AC                   1
DF                   1
PB                   1
Jaraguá do sul       1
Name: count, dtype: int64


In [91]:
df_leads_l34_forms['escolaridade_original'] = df_leads_l34_forms['escolaridade']
df_leads_l34_forms['escolaridade'] = df_leads_l34_forms['escolaridade_original'].apply(normalizar_escolaridade)

df_leads_l34_forms['escolaridade'].value_counts()

escolaridade
médio completo            327
superior completo          63
fundamental completo       47
superior incompleto        18
técnico                     9
médio incompleto            6
fundamental incompleto      3
Name: count, dtype: int64

In [92]:
df_leads_l34_forms['estado_civil_original'] = df_leads_l34_forms['estado_civil']
df_leads_l34_forms['estado_civil'] = df_leads_l34_forms['estado_civil_original'].apply(normalizar_estado_civil)

df_leads_l34_forms['estado_civil'].value_counts()

estado_civil
casado(a)                       187
solteiro(a)                     161
união estável                    65
divorciado(a) ou separado(a)     58
Name: count, dtype: int64

In [93]:
df_leads_l34_forms['dificuldade_original'] = df_leads_l34_forms['dificuldade']
df_leads_l34_forms['dificuldade'] = df_leads_l34_forms['dificuldade_original'].apply(limpar_dificuldade_simples)

df_leads_l34_forms['dificuldade'].value_counts()

dificuldade
financeiro / dinheiro    252
falta de tempo            82
falta de base escolar     65
idade                     49
falta de oportunidade      3
Name: count, dtype: int64

In [94]:
# Excluir colunas que terminam com "_original"
df_leads_l34_forms.drop(
    columns=[col for col in df_leads_l34_forms.columns if col.endswith('_original')],
    inplace=True
)

In [95]:
colunas_excluir = ["data", "email", "whatsapp", "nome"]

for coluna in df_leads_l34_forms.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34_forms[coluna].value_counts(dropna=False))


Coluna: estado
estado
SP                 294
MG                  45
BA                  29
PE                  19
RS                  18
PR                  14
RJ                  13
SC                  12
AL                  11
MA                   5
ES                   3
CE                   3
GO                   3
PA                   2
RN                   2
TO                   1
AC                   1
DF                   1
PB                   1
Jaraguá do sul       1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        202
46 - 55 anos        145
26 - 35 anos        108
Até 25 anos          19
Acima de 56 anos      4
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            327
superior completo          63
fundamental completo       47
superior incompleto        18
técnico                     9
médio incompleto            6
None                        5
fundamental incompleto      3
Name: count, dtype: int64

Coluna: renda
renda
De

In [96]:
parquet_path = Path.cwd().parent / "dados" / "leads_l34.parquet"
df_leads_l34 = pd.read_parquet(parquet_path)

In [97]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   id                         5572 non-null   object
 1   nome                       5572 non-null   object
 2   email                      5572 non-null   object
 3   telefone                   5572 non-null   object
 4   data inscrição lançamento  5572 non-null   object
 5   estado                     4568 non-null   object
 6   idade                      4510 non-null   object
 7   escolaridade               4513 non-null   object
 8   renda                      4535 non-null   object
 9   estado civil               4520 non-null   object
 10  filhos                     4540 non-null   object
 11  escolheu profissão         4557 non-null   object
 12  dificuldade                4276 non-null   object
 13  email captação             4568 non-null   object
 14  telefone

In [98]:
colunas_para_atualizar = [
    'estado', 'idade', 'escolaridade', 'renda',
    'estado_civil', 'filhos', 'escolheu_profissao', 'dificuldade'
]

# Remove duplicatas com base no e-mail (mantendo o mais recente ou o primeiro)
df_leads_l34 = df_leads_l34.drop_duplicates(subset='email', keep='last')
df_atualizacao = df_leads_l34_forms[['email'] + colunas_para_atualizar].drop_duplicates(subset='email', keep='last')

# Define o email como índice
df_leads_l34.set_index('email', inplace=True)
df_atualizacao.set_index('email', inplace=True)

# Atualiza os dados
df_leads_l34.update(df_atualizacao)

# (Opcional) Resetar índice se necessário
df_leads_l34.reset_index(inplace=True)

In [99]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   email                      5572 non-null   object
 1   id                         5572 non-null   object
 2   nome                       5572 non-null   object
 3   telefone                   5572 non-null   object
 4   data inscrição lançamento  5572 non-null   object
 5   estado                     4572 non-null   object
 6   idade                      4516 non-null   object
 7   escolaridade               4520 non-null   object
 8   renda                      4543 non-null   object
 9   estado civil               4520 non-null   object
 10  filhos                     4544 non-null   object
 11  escolheu profissão         4557 non-null   object
 12  dificuldade                4294 non-null   object
 13  email captação             4568 non-null   object
 14  telefone

In [100]:
df_leads_l34 = df_leads_l34[df_leads_l34['email captação'].notna()].reset_index(drop=True)

In [101]:
df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4568 entries, 0 to 4567
Data columns (total 20 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   email                      4568 non-null   object
 1   id                         4568 non-null   object
 2   nome                       4568 non-null   object
 3   telefone                   4568 non-null   object
 4   data inscrição lançamento  4568 non-null   object
 5   estado                     4568 non-null   object
 6   idade                      4512 non-null   object
 7   escolaridade               4516 non-null   object
 8   renda                      4539 non-null   object
 9   estado civil               4520 non-null   object
 10  filhos                     4540 non-null   object
 11  escolheu profissão         4557 non-null   object
 12  dificuldade                4290 non-null   object
 13  email captação             4568 non-null   object
 14  telefone

In [102]:
df_leads_l34 = df_leads_l34.drop(columns=['id', 'email', 'telefone', 'nome'])

# Renomear colunas do df_leads_google
colunas_renomeadas = {
    'data inscrição lançamento': 'data',
    'email captação': 'email',
    'telefone captação': 'whatsapp',
    'Qual estado você reside?': 'estado',
    'Qual sua faixa de idade?': 'idade',
    'Qual seu nível de escolaridade?': 'escolaridade',
    'Qual sua faixa salarial atualmente?': 'renda',
    'estado civil': 'estado_civil',
    'Você tem filhos?': 'filhos',
    'escolheu profissão': 'escolheu_profissao'
}

df_leads_l34.rename(columns=colunas_renomeadas, inplace=True)

# Normalizar e deduplicar e-mails
if 'email' in df_leads_l34.columns:
    df_leads_l34['email'] = df_leads_l34['email'].astype(str).str.lower().str.strip()

df_leads_l34["lancamentos"] = "L34"

df_leads_l34["comprou"] = df_leads_l34["email"].isin(df_compradores["email"]).astype(int)

df_leads_l34.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4568 entries, 0 to 4567
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                4568 non-null   object
 1   estado              4568 non-null   object
 2   idade               4512 non-null   object
 3   escolaridade        4516 non-null   object
 4   renda               4539 non-null   object
 5   estado_civil        4520 non-null   object
 6   filhos              4540 non-null   object
 7   escolheu_profissao  4557 non-null   object
 8   dificuldade         4290 non-null   object
 9   email               4568 non-null   object
 10  whatsapp            4566 non-null   object
 11  utm_source          4465 non-null   object
 12  utm_campaign        4162 non-null   object
 13  utm_medium          4162 non-null   object
 14  utm_content         4098 non-null   object
 15  utm_term            3939 non-null   object
 16  lancamentos         4568

In [103]:
df_leads_l34["renda"] = (
    df_leads_l34["renda"]
    .replace({
        "De R$ 1.000,00 a R$ 3.000,00": "De 1.000 a 3.000",
        "De R$ 3.000,00 a R$ 5.000,00": "De 3.000 a 5.000",
        "Até R$ 1.000,00": "Até 1.000",
        "Não estou trabalhando no momento": "Desempregado",
        "Acima de R$ 5.000,00": "Acima de 5.000"
    })
)

In [104]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads_l34.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34[coluna].value_counts(dropna=False))


Coluna: estado
estado
São Paulo             2098
Outro                 1300
Minas Gerais           401
SP                     254
Paraná                 220
Rio de Janeiro         116
MG                      38
BA                      26
Mato Grosso do Sul      23
RS                      18
PE                      17
RJ                      12
SC                      11
PR                      10
AL                       7
ES                       3
MA                       3
CE                       3
PA                       2
AC                       1
TO                       1
RN                       1
GO                       1
DF                       1
Jaraguá do sul           1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        2073
46 - 55 anos        1153
26 - 35 anos        1065
Até 25 anos          187
None                  56
Acima de 56 anos      34
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
Ensino Médio Completo          2961
Ensino S

In [105]:
# Aplicar a função
df_leads_l34['estado'] = df_leads_l34['estado'].apply(normalizar_estado)

# Verificar resultado final
print(df_leads_l34['estado'].value_counts(dropna=False))

estado
SP                 2352
Outro              1300
MG                  439
PR                  230
RJ                  128
BA                   26
MA                   26
RS                   18
PE                   17
SC                   11
AL                    7
CE                    3
ES                    3
PA                    2
TO                    1
AC                    1
RN                    1
GO                    1
DF                    1
Jaraguá do sul        1
Name: count, dtype: int64


In [106]:
for col in features:
    if col in df_leads_l34.columns:
        df_leads_l34[col] = df_leads[col].astype(str).str.strip().str.lower()

# Remove linhas com None ou NaN em qualquer uma dessas colunas
df_leads_l34 = df_leads_l34.dropna(subset=features).reset_index(drop=True)

In [107]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads_l34.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads_l34[coluna].value_counts(dropna=False))


Coluna: estado
estado
SP                 2352
Outro              1300
MG                  439
PR                  230
RJ                  128
BA                   26
MA                   26
RS                   18
PE                   17
SC                   11
AL                    7
CE                    3
ES                    3
PA                    2
TO                    1
AC                    1
RN                    1
GO                    1
DF                    1
Jaraguá do sul        1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        1750
26 - 35 anos        1390
46 - 55 anos        1020
até 25 anos          323
acima de 56 anos      85
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            3296
superior completo          808
fundamental completo       409
superior incompleto         40
técnico                      9
fundamental incompleto       3
médio incompleto             3
Name: count, dtype: int64

Coluna: renda
renda


# Merge `df_leads` com `df_lead_novo`

In [108]:
# Recarregue os dois DataFrames originais se possível (ou copie)
df_leads = df_leads.copy()
df_leads_l34 = df_leads_l34.copy()

# Junte corretamente
df_leads = pd.concat([df_leads, df_leads_l34], ignore_index=True)
print("Total:", len(df_leads))

Total: 15431


In [109]:
colunas_excluir = ["data", "email", "whatsapp"]

for coluna in df_leads.columns:
    if coluna not in colunas_excluir:
        print(f"\nColuna: {coluna}")
        print(df_leads[coluna].value_counts(dropna=False))


Coluna: lancamentos
lancamentos
L32    4875
L34    4568
L31    2657
L33    2461
L30     352
L28     275
L29     243
Name: count, dtype: int64

Coluna: estado
estado
SP                 10358
RJ                  1621
Outro               1516
MG                  1474
PR                   271
MA                    61
BA                    39
RS                    22
PE                    18
SC                    14
AL                     9
GO                     9
CE                     5
ES                     3
DF                     2
RN                     2
PA                     2
RR                     1
AM                     1
AC                     1
TO                     1
Jaraguá do sul         1
Name: count, dtype: int64

Coluna: idade
idade
36 - 45 anos        5934
26 - 35 anos        4760
46 - 55 anos        3368
até 25 anos         1101
acima de 56 anos     268
Name: count, dtype: int64

Coluna: escolaridade
escolaridade
médio completo            11259
superior completo  

In [110]:
df_leads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15431 entries, 0 to 15430
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data                15431 non-null  object
 1   lancamentos         15431 non-null  object
 2   email               15431 non-null  object
 3   whatsapp            15429 non-null  object
 4   estado              15431 non-null  object
 5   idade               15431 non-null  object
 6   escolaridade        15431 non-null  object
 7   renda               15431 non-null  object
 8   estado_civil        15431 non-null  object
 9   filhos              15431 non-null  object
 10  escolheu_profissao  15431 non-null  object
 11  dificuldade         15431 non-null  object
 12  nome                870 non-null    object
 13  comprou             15431 non-null  int32 
 14  utm_source          15282 non-null  object
 15  utm_medium          14852 non-null  object
 16  utm_campaign        15

In [111]:
df_leads['data'] = pd.to_datetime(df_leads['data'], errors='coerce')
df_alunos['data'] = pd.to_datetime(df_alunos['data'], errors='coerce')

In [112]:
output_dir = Path.cwd().parent / "dados"

df_leads.to_parquet(output_dir / "leads.parquet", index=False)
df_alunos.to_parquet(output_dir / "alunos.parquet", index=False)

In [113]:
# Gerar timestamp com fuso horário de SP
fuso_brasil = pytz.timezone("America/Sao_Paulo")
agora_brasil = datetime.now(fuso_brasil)

# Caminho seguro para salvar
config_path = Path.cwd().parent / "config" / "ultima_atualizacao.txt"
with open(config_path, "w") as f:
    f.write(agora_brasil.strftime("%Y-%m-%d %H:%M:%S"))

In [114]:
try:
    caminho_data = Path.cwd().parent / "config" / "ultima_atualizacao.txt"
    with open(caminho_data, "r") as f:
        texto = f.read()
        data_atualizacao = datetime.strptime(texto, "%Y-%m-%d %H:%M:%S")
        data_atualizacao_formatada = data_atualizacao.strftime("%d/%m/%Y %H:%M")
except Exception:
    data_atualizacao_formatada = "Desconhecida"

In [115]:
f"**Última atualização:** {data_atualizacao_formatada}"

'**Última atualização:** 21/05/2025 17:03'